In [95]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
import re
from cltk.tokenize.word import WordTokenizer
import string

In [2]:
quran_df = pd.read_csv('../data/quran_data.csv')
hadith_df = pd.read_csv('../data/all_hadiths_clean.csv')

In [18]:
quran_df.head()

,Surah Name,Surah Number,Verse Number,Verse Text
0,الفاتحة,1,1,بِسْمِ اللَّهِ الرَّحْمَنِ الرَّحِيمِ
1,الفاتحة,1,2,الْحَمْدُ لِلَّهِ رَبِّ الْعَالَمِينَ
2,الفاتحة,1,3,الرَّحْمَنِ الرَّحِيمِ
3,الفاتحة,1,4,مَالِكِ يَوْمِ الدِّينِ
4,الفاتحة,1,5,إِيَّاكَ نَعْبُدُ وَإِيَّاكَ نَسْتَعِينُ


In [96]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)


def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

def remove_weird_chars(text):
    return text.replace('\u200f', '')

In [97]:
cleaning_funcs = [normalize_arabic, remove_diacritics, remove_punctuations, remove_weird_chars]

In [106]:
documents = quran_df['Verse Text'].dropna()

In [107]:
clean_documents = []
tokenizer = WordTokenizer('arabic')

for document in documents:

    for f in cleaning_funcs:
        document = f(document)

    tokens = tokenizer.tokenize(document)
    clean_documents.append(tokens)

In [109]:
len(clean_documents)

6236

In [153]:
str(clean_documents[0])

"['بسم', 'الله', 'الرحمن', 'الرحيم']"

In [154]:
clean_documents2 = [' '.join(x) for x in clean_documents]

In [155]:
clean_documents2[0]

'بسم الله الرحمن الرحيم'

In [145]:
clean_documents[0]

['بسم', 'الله', 'الرحمن', 'الرحيم']

In [156]:
vectorizer = TfidfVectorizer()

In [157]:
X = vectorizer.fit_transform(clean_documents2)

In [158]:
print(vectorizer.get_feature_names())

['اءت', 'اءتمن', 'اءتنا', 'اءتوا', 'اءتوني', 'اءتيا', 'اءذا', 'اءذن', 'اءفكا', 'اءمه', 'اءن', 'اءنا', 'اءنبءكم', 'اءنكم', 'ااتخذ', 'ااذا', 'اارباب', 'ااسجد', 'ااسلمتم', 'ااشفقتم', 'ااشكر', 'ااعجمي', 'ااقررتم', 'االد', 'االقي', 'االه', 'االهتنا', 'اامنتم', 'اانا', 'اانت', 'اانتم', 'اانذرتهم', 'اانزل', 'اانك', 'ابا', 'اباء', 'اباءك', 'اباءكم', 'اباءنا', 'اباءهم', 'اباءهن', 'اباءي', 'ابابيل', 'اباكم', 'ابالله', 'ابانا', 'اباه', 'اباهم', 'ابت', 'ابتدعوها', 'ابتغاء', 'ابتغوا', 'ابتغي', 'ابتغيت', 'ابتلاه', 'ابتلي', 'ابحر', 'ابدا', 'ابدله', 'ابرء', 'ابراهيم', 'ابرح', 'ابرموا', 'ابسلوا', 'ابشر', 'ابشرا', 'ابشرتموني', 'ابصار', 'ابصاركم', 'ابصارنا', 'ابصارها', 'ابصارهم', 'ابصارهن', 'ابصر', 'ابصرنا', 'ابعث', 'ابغي', 'ابغيكم', 'ابق', 'ابقي', 'ابكارا', 'ابكم', 'ابلعي', 'ابلغ', 'ابلغتكم', 'ابلغكم', 'ابلغه', 'ابلغوا', 'ابليس', 'ابن', 'ابناء', 'ابناءكم', 'ابناءنا', 'ابناءهم', 'ابناءهن', 'ابنت', 'ابنتي', 'ابنك', 'ابنه', 'ابنوا', 'ابني', 'ابواب', 'ابوابا', 'ابوابها', 'ابواه', 'ابوك', 'ابوهم', 'ابوهما', 

In [159]:

feature_names = vectorizer.get_feature_names()
for col in X.nonzero()[1]:
    print (feature_names[col], ' - ', X[0, col])

بسم  -  0.6851975525542379
الله  -  0.19531327476066437
الرحمن  -  0.48482660154849483
الرحيم  -  0.5072476762952044
الحمد  -  0.0
لله  -  0.0
رب  -  0.0
العالمين  -  0.0
الرحمن  -  0.48482660154849483
الرحيم  -  0.5072476762952044
مالك  -  0.0
يوم  -  0.0
الدين  -  0.0
اياك  -  0.0
نعبد  -  0.0
واياك  -  0.0
نستعين  -  0.0
اهدنا  -  0.0
الصراط  -  0.0
المستقيم  -  0.0
صراط  -  0.0
الذين  -  0.0
انعمت  -  0.0
عليهم  -  0.0
غير  -  0.0
المغضوب  -  0.0
ولا  -  0.0
الضالين  -  0.0
الم  -  0.0
ذلك  -  0.0
الكتاب  -  0.0
لا  -  0.0
ريب  -  0.0
فيه  -  0.0
هدي  -  0.0
للمتقين  -  0.0
الذين  -  0.0
يءمنون  -  0.0
بالغيب  -  0.0
ويقيمون  -  0.0
الصلاه  -  0.0
ومما  -  0.0
رزقناهم  -  0.0
ينفقون  -  0.0
يءمنون  -  0.0
والذين  -  0.0
بما  -  0.0
انزل  -  0.0
اليك  -  0.0
وما  -  0.0
من  -  0.0
قبلك  -  0.0
وبالاخره  -  0.0
هم  -  0.0
يوقنون  -  0.0
هدي  -  0.0
من  -  0.0
هم  -  0.0
اولءك  -  0.0
علي  -  0.0
ربهم  -  0.0
واولءك  -  0.0
المفلحون  -  0.0
الذين  -  0.0
عليهم  -  0.0
لا  -  0.0
يءمنو

ثمنا  -  0.0
قليلا  -  0.0
فاتقون  -  0.0
ولا  -  0.0
وانتم  -  0.0
تعلمون  -  0.0
الحق  -  0.0
تلبسوا  -  0.0
بالباطل  -  0.0
وتكتموا  -  0.0
الصلاه  -  0.0
واتوا  -  0.0
واقيموا  -  0.0
الزكاه  -  0.0
واركعوا  -  0.0
مع  -  0.0
الراكعين  -  0.0
الكتاب  -  0.0
الناس  -  0.0
وانتم  -  0.0
اتامرون  -  0.0
بالبر  -  0.0
وتنسون  -  0.0
انفسكم  -  0.0
تتلون  -  0.0
افلا  -  0.0
تعقلون  -  0.0
علي  -  0.0
الا  -  0.0
واستعينوا  -  0.0
بالصبر  -  0.0
والصلاه  -  0.0
وانها  -  0.0
لكبيره  -  0.0
الخاشعين  -  0.0
الذين  -  0.0
ربهم  -  0.0
انهم  -  0.0
اليه  -  0.0
يظنون  -  0.0
ملاقو  -  0.0
وانهم  -  0.0
راجعون  -  0.0
العالمين  -  0.0
انعمت  -  0.0
علي  -  0.0
يا  -  0.0
التي  -  0.0
بني  -  0.0
اسراءيل  -  0.0
اذكروا  -  0.0
نعمتي  -  0.0
عليكم  -  0.0
واني  -  0.0
فضلتكم  -  0.0
ولا  -  0.0
لا  -  0.0
هم  -  0.0
منها  -  0.0
واتقوا  -  0.0
يوما  -  0.0
تجزي  -  0.0
نفس  -  0.0
عن  -  0.0
شيءا  -  0.0
يقبل  -  0.0
شفاعه  -  0.0
يءخذ  -  0.0
عدل  -  0.0
ينصرون  -  0.0
من  -  0.0
عظيم  -  0.

تقولون  -  0.0
من  -  0.0
هم  -  0.0
به  -  0.0
النار  -  0.0
فيها  -  0.0
خالدون  -  0.0
اصحاب  -  0.0
بلي  -  0.0
كسب  -  0.0
سيءه  -  0.0
واحاطت  -  0.0
خطيءته  -  0.0
فاولءك  -  0.0
والذين  -  0.0
هم  -  0.0
اولءك  -  0.0
امنوا  -  0.0
وعملوا  -  0.0
الصالحات  -  0.0
فيها  -  0.0
خالدون  -  0.0
الجنه  -  0.0
اصحاب  -  0.0
الله  -  0.19531327476066437
لا  -  0.0
الصلاه  -  0.0
الا  -  0.0
وانتم  -  0.0
واتوا  -  0.0
ثم  -  0.0
واذ  -  0.0
بني  -  0.0
اسراءيل  -  0.0
قليلا  -  0.0
واقيموا  -  0.0
الزكاه  -  0.0
وقولوا  -  0.0
اخذنا  -  0.0
توليتم  -  0.0
منكم  -  0.0
ميثاق  -  0.0
تعبدون  -  0.0
وبالوالدين  -  0.0
احسانا  -  0.0
وذي  -  0.0
القربي  -  0.0
واليتامي  -  0.0
والمساكين  -  0.0
للناس  -  0.0
حسنا  -  0.0
معرضون  -  0.0
ولا  -  0.0
لا  -  0.0
من  -  0.0
وانتم  -  0.0
ثم  -  0.0
واذ  -  0.0
انفسكم  -  0.0
اخذنا  -  0.0
ميثاقكم  -  0.0
تسفكون  -  0.0
دماءكم  -  0.0
تخرجون  -  0.0
دياركم  -  0.0
اقررتم  -  0.0
تشهدون  -  0.0
الله  -  0.19531327476066437
عليهم  -  0.0
ذلك  -  

عليم  -  0.0
ولله  -  0.0
المشرق  -  0.0
والمغرب  -  0.0
فاينما  -  0.0
تولوا  -  0.0
فثم  -  0.0
وجه  -  0.0
واسع  -  0.0
الله  -  0.19531327476066437
في  -  0.0
ما  -  0.0
كل  -  0.0
السماوات  -  0.0
والارض  -  0.0
وقالوا  -  0.0
بل  -  0.0
له  -  0.0
اتخذ  -  0.0
ولدا  -  0.0
سبحانه  -  0.0
قانتون  -  0.0
يقول  -  0.0
واذا  -  0.0
السماوات  -  0.0
والارض  -  0.0
له  -  0.0
بديع  -  0.0
قضي  -  0.0
امرا  -  0.0
فانما  -  0.0
كن  -  0.0
فيكون  -  0.0
الله  -  0.19531327476066437
الذين  -  0.0
لا  -  0.0
من  -  0.0
يوقنون  -  0.0
قلوبهم  -  0.0
يعلمون  -  0.0
او  -  0.0
قال  -  0.0
قد  -  0.0
كذلك  -  0.0
ايه  -  0.0
مثل  -  0.0
قولهم  -  0.0
وقال  -  0.0
لولا  -  0.0
يكلمنا  -  0.0
تاتينا  -  0.0
قبلهم  -  0.0
تشابهت  -  0.0
بينا  -  0.0
الايات  -  0.0
لقوم  -  0.0
ولا  -  0.0
انا  -  0.0
اصحاب  -  0.0
عن  -  0.0
بالحق  -  0.0
ارسلناك  -  0.0
بشيرا  -  0.0
ونذيرا  -  0.0
تسال  -  0.0
الجحيم  -  0.0
الله  -  0.19531327476066437
ولا  -  0.0
هدي  -  0.0
من  -  0.0
ان  -  0.0
الذي  -  0.0

عليهم  -  0.0
وما  -  0.0
من  -  0.0
هم  -  0.0
ان  -  0.0
كما  -  0.0
النار  -  0.0
لنا  -  0.0
كذلك  -  0.0
منهم  -  0.0
لو  -  0.0
وقال  -  0.0
منا  -  0.0
اتبعوا  -  0.0
كره  -  0.0
فنتبرا  -  0.0
تبرءوا  -  0.0
يريهم  -  0.0
اعمالهم  -  0.0
حسرات  -  0.0
بخارجين  -  0.0
ولا  -  0.0
الناس  -  0.0
في  -  0.0
الارض  -  0.0
يا  -  0.0
ايها  -  0.0
لكم  -  0.0
مما  -  0.0
انه  -  0.0
الشيطان  -  0.0
عدو  -  0.0
كلوا  -  0.0
حلالا  -  0.0
طيبا  -  0.0
تتبعوا  -  0.0
خطوات  -  0.0
مبين  -  0.0
الله  -  0.19531327476066437
لا  -  0.0
علي  -  0.0
انما  -  0.0
ما  -  0.0
تعلمون  -  0.0
وان  -  0.0
يامركم  -  0.0
تقولوا  -  0.0
بالسوء  -  0.0
والفحشاء  -  0.0
الله  -  0.19531327476066437
ولا  -  0.0
لا  -  0.0
انزل  -  0.0
واذا  -  0.0
قيل  -  0.0
لهم  -  0.0
قالوا  -  0.0
ما  -  0.0
عليه  -  0.0
شيءا  -  0.0
كان  -  0.0
بل  -  0.0
يعقلون  -  0.0
اتبعوا  -  0.0
نتبع  -  0.0
الفينا  -  0.0
اباءنا  -  0.0
اولو  -  0.0
اباءهم  -  0.0
يهتدون  -  0.0
الذين  -  0.0
لا  -  0.0
بما  -  0.0
كفروا  - 

...

تسعه  -  0.0
الله  -  0.19531327476066437
الذين  -  0.0
ولا  -  0.0
الكتاب  -  0.0
وما  -  0.0
من  -  0.0
كفروا  -  0.0
قلوبهم  -  0.0
امنوا  -  0.0
الا  -  0.0
في  -  0.0
مرض  -  0.0
النار  -  0.0
مثلا  -  0.0
ماذا  -  0.0
اراد  -  0.0
بهذا  -  0.0
يضل  -  0.0
ويهدي  -  0.0
هو  -  0.0
ربك  -  0.0
اصحاب  -  0.0
هي  -  0.0
كذلك  -  0.0
يعلم  -  0.0
للذين  -  0.0
يشاء  -  0.0
اوتوا  -  0.0
فتنه  -  0.0
جعلنا  -  0.0
والكافرون  -  0.0
والمءمنون  -  0.0
ايمانا  -  0.0
ذكري  -  0.0
ملاءكه  -  0.0
جنود  -  0.0
للبشر  -  0.0
عدتهم  -  0.0
ليستيقن  -  0.0
ويزداد  -  0.0
يرتاب  -  0.0
وليقول  -  0.0
كلا  -  0.0
والقمر  -  0.0
اذ  -  0.0
ادبر  -  0.0
والليل  -  0.0
اذا  -  0.0
والصبح  -  0.0
اسفر  -  0.0
انها  -  0.0
الكبر  -  0.0
لاحدي  -  0.0
نذيرا  -  0.0
للبشر  -  0.0
ان  -  0.0
او  -  0.0
شاء  -  0.0
منكم  -  0.0
لمن  -  0.0
يتقدم  -  0.0
يتاخر  -  0.0
بما  -  0.0
كل  -  0.0
نفس  -  0.0
كسبت  -  0.0
رهينه  -  0.0
الا  -  0.0
اصحاب  -  0.0
اليمين  -  0.0
في  -  0.0
جنات  -  0.0
يتساءلون  -  

ابوابا  -  0.0
الجبال  -  0.0
فكانت  -  0.0
وسيرت  -  0.0
سرابا  -  0.0
ان  -  0.0
كانت  -  0.0
جهنم  -  0.0
مرصادا  -  0.0
للطاغين  -  0.0
مابا  -  0.0
فيها  -  0.0
لابثين  -  0.0
احقابا  -  0.0
ولا  -  0.0
لا  -  0.0
فيها  -  0.0
بردا  -  0.0
يذوقون  -  0.0
شرابا  -  0.0
الا  -  0.0
حميما  -  0.0
وغساقا  -  0.0
جزاء  -  0.0
وفاقا  -  0.0
لا  -  0.0
كانوا  -  0.0
انهم  -  0.0
يرجون  -  0.0
حسابا  -  0.0
وكذبوا  -  0.0
باياتنا  -  0.0
كذابا  -  0.0
شيء  -  0.0
كتابا  -  0.0
وكل  -  0.0
احصيناه  -  0.0
الا  -  0.0
فلن  -  0.0
عذابا  -  0.0
فذوقوا  -  0.0
نزيدكم  -  0.0
للمتقين  -  0.0
ان  -  0.0
مفازا  -  0.0
حداءق  -  0.0
واعنابا  -  0.0
اترابا  -  0.0
وكواعب  -  0.0
وكاسا  -  0.0
دهاقا  -  0.0
ولا  -  0.0
لا  -  0.0
فيها  -  0.0
يسمعون  -  0.0
لغوا  -  0.0
كذابا  -  0.0
من  -  0.0
ربك  -  0.0
جزاء  -  0.0
عطاء  -  0.0
حسابا  -  0.0
الرحمن  -  0.48482660154849483
رب  -  0.0
لا  -  0.0
وما  -  0.0
السماوات  -  0.0
والارض  -  0.0
منه  -  0.0
بينهما  -  0.0
يملكون  -  0.0
خطابا  -  0.0
ال

مدت  -  0.0
ما  -  0.0
فيها  -  0.0
والقت  -  0.0
وتخلت  -  0.0
واذنت  -  0.0
لربها  -  0.0
وحقت  -  0.0
الي  -  0.0
يا  -  0.0
ايها  -  0.0
ربك  -  0.0
انك  -  0.0
الانسان  -  0.0
كادح  -  0.0
كدحا  -  0.0
فملاقيه  -  0.0
من  -  0.0
فاما  -  0.0
اوتي  -  0.0
كتابه  -  0.0
بيمينه  -  0.0
فسوف  -  0.0
يسيرا  -  0.0
حسابا  -  0.0
يحاسب  -  0.0
الي  -  0.0
اهله  -  0.0
وينقلب  -  0.0
مسرورا  -  0.0
من  -  0.0
واما  -  0.0
وراء  -  0.0
اوتي  -  0.0
كتابه  -  0.0
ظهره  -  0.0
يدعو  -  0.0
فسوف  -  0.0
ثبورا  -  0.0
سعيرا  -  0.0
ويصلي  -  0.0
في  -  0.0
انه  -  0.0
كان  -  0.0
اهله  -  0.0
مسرورا  -  0.0
ان  -  0.0
انه  -  0.0
لن  -  0.0
ظن  -  0.0
يحور  -  0.0
ان  -  0.0
به  -  0.0
ربه  -  0.0
كان  -  0.0
بلي  -  0.0
بصيرا  -  0.0
فلا  -  0.0
اقسم  -  0.0
بالشفق  -  0.0
وما  -  0.0
والليل  -  0.0
وسق  -  0.0
اذا  -  0.0
والقمر  -  0.0
اتسق  -  0.0
عن  -  0.0
لتركبن  -  0.0
طبقا  -  0.0
طبق  -  0.0
لا  -  0.0
يءمنون  -  0.0
لهم  -  0.0
فما  -  0.0
عليهم  -  0.0
لا  -  0.0
واذا  -  0.0
القرا

يءتي  -  0.0
ماله  -  0.0
يتزكي  -  0.0
وما  -  0.0
من  -  0.0
تجزي  -  0.0
عنده  -  0.0
نعمه  -  0.0
لاحد  -  0.0
الا  -  0.0
ربه  -  0.0
وجه  -  0.0
ابتغاء  -  0.0
الاعلي  -  0.0
يرضي  -  0.0
ولسوف  -  0.0
والضحي  -  0.0
اذا  -  0.0
والليل  -  0.0
سجي  -  0.0
وما  -  0.0
ما  -  0.0
ربك  -  0.0
ودعك  -  0.0
قلي  -  0.0
من  -  0.0
لك  -  0.0
خير  -  0.0
وللاخره  -  0.0
الاولي  -  0.0
ربك  -  0.0
ولسوف  -  0.0
يعطيك  -  0.0
فترضي  -  0.0
الم  -  0.0
يتيما  -  0.0
يجدك  -  0.0
فاوي  -  0.0
فهدي  -  0.0
ووجدك  -  0.0
ضالا  -  0.0
ووجدك  -  0.0
عاءلا  -  0.0
فاغني  -  0.0
فلا  -  0.0
فاما  -  0.0
اليتيم  -  0.0
تقهر  -  0.0
فلا  -  0.0
واما  -  0.0
الساءل  -  0.0
تنهر  -  0.0
واما  -  0.0
ربك  -  0.0
بنعمه  -  0.0
فحدث  -  0.0
الم  -  0.0
لك  -  0.0
صدرك  -  0.0
نشرح  -  0.0
عنك  -  0.0
ووضعنا  -  0.0
وزرك  -  0.0
الذي  -  0.0
انقض  -  0.0
ظهرك  -  0.0
لك  -  0.0
ورفعنا  -  0.0
ذكرك  -  0.0
فان  -  0.0
مع  -  0.0
العسر  -  0.0
يسرا  -  0.0
ان  -  0.0
مع  -  0.0
العسر  -  0.0
يسرا  -  0.0
ف

يدا  -  0.0
لهب  -  0.0
وما  -  0.0
ما  -  0.0
كسب  -  0.0
ماله  -  0.0
عنه  -  0.0
اغني  -  0.0
نارا  -  0.0
ذات  -  0.0
لهب  -  0.0
سيصلي  -  0.0
وامراته  -  0.0
حماله  -  0.0
الحطب  -  0.0
من  -  0.0
في  -  0.0
حبل  -  0.0
جيدها  -  0.0
مسد  -  0.0
الله  -  0.19531327476066437
هو  -  0.0
قل  -  0.0
احد  -  0.0
الله  -  0.19531327476066437
الصمد  -  0.0
لم  -  0.0
ولم  -  0.0
يلد  -  0.0
يولد  -  0.0
احد  -  0.0
له  -  0.0
يكن  -  0.0
ولم  -  0.0
كفوا  -  0.0
اعوذ  -  0.0
قل  -  0.0
برب  -  0.0
الفلق  -  0.0
من  -  0.0
ما  -  0.0
خلق  -  0.0
شر  -  0.0
ومن  -  0.0
اذا  -  0.0
شر  -  0.0
غاسق  -  0.0
وقب  -  0.0
ومن  -  0.0
في  -  0.0
شر  -  0.0
النفاثات  -  0.0
العقد  -  0.0
ومن  -  0.0
اذا  -  0.0
شر  -  0.0
حاسد  -  0.0
حسد  -  0.0
الناس  -  0.0
اعوذ  -  0.0
قل  -  0.0
برب  -  0.0
الناس  -  0.0
ملك  -  0.0
الناس  -  0.0
اله  -  0.0
من  -  0.0
شر  -  0.0
الوسواس  -  0.0
الخناس  -  0.0
الناس  -  0.0
في  -  0.0
الذي  -  0.0
صدور  -  0.0
يوسوس  -  0.0
من  -  0.0
الجنه  -  0.0
والناس  -

In [65]:
 def computeReviewTFDict(review):
    """ Returns a tf dictionary for each review whose keys are all 
    the unique words in the review and whose values are their 
    corresponding tf.
    """
    #Counts the number of times the word appears in review
    reviewTFDict = {}
    for word in review:
        if word in reviewTFDict:
            reviewTFDict[word] += 1
        else:
            reviewTFDict[word] = 1
    #Computes tf for each word           
    for word in reviewTFDict:
        reviewTFDict[word] = reviewTFDict[word] / len(review)
    return reviewTFDict

In [78]:
tfDict = computeReviewTFDict(a)

In [79]:
tfDict[0]

KeyError: 0

In [7]:
vectorizer = TfidfVectorizer()

In [8]:
X = vectorizer.fit_transform(corpus)

In [9]:
print(vectorizer.get_feature_names())

['and', 'document', 'first', 'is', 'one', 'second', 'the', 'third', 'this']


In [10]:
print(X.shape)

(4, 9)
